In [1]:
# Povm

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_base import ProjectedGradientDescentBase, ProjectedGradientDescentBaseOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_objects = [state_x0, state_y0, state_z0, state_z1]

In [6]:
# Case 1: 軸上
# a0, a1, a2, a3 = 1, 1, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 2: 軸から一番離れた表面
a0, a1, a2, a3 = 1, 1 / np.sqrt(2), 0, 1 / np.sqrt(2)
m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 3: 原点
# a0, a1, a2, a3 = 1, 0, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

In [7]:
num_data = [100, 1000, 10000]
n_rep = 10
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=True, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=False, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(4),
    WeightedRelativeEntropy(8),
    WeightedProbabilityBasedSquaredError(4),
    WeightedProbabilityBasedSquaredError(8),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption(),
    WeightedRelativeEntropyOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       loss=loss,
       loss_option=loss_option,
       algo=algo,
       algo_option=algo_option,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 20%|██        | 2/10 [00:00<00:00, 12.35it/s]Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LinearEstimator
 10%|█         | 1/10 [00:00<00:00,  9.09it/s]elapsed_time:0.01723326841990153[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LinearEstimator
 10%|█         | 1/10 [00:00<00:01,  5.81it/s]elapsed_time:0.01824998458226522[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.0260780135790507[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.027833290894826255[min]

Case 4: Maximum-Likelihood(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LossMinimizationEstimator
**

In [8]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
    estimator_list=estimator_list,
)

LinearEstimator(True)=[0.03203786692715928, 0.0028377041786821627, 0.00024584712280146745]
LinearEstimator(False)=[0.020277866927159285, 0.0035415645042405457, 0.0003277628080438488]
ProjectedLinearEstimator(True)=[0.026550064138798422, 0.0021114438615901223, 0.0002675468409009211]
ProjectedLinearEstimator(False)=[0.01535863010678103, 0.0021758248129599765, 0.0002449240012118075]
Maximum-Likelihood(True)=[0.01877887382799106, 0.0012239607956772345, 0.000233965320715483]
Maximum-Likelihood(False)=[0.016422667046116113, 0.002881712135450174, 0.00021016488180236893]
Least Squares(True)=[0.01615483569034127, 0.0013013632180900504, 0.0001426711328080634]
Least Squares(False)=[0.015407658029423215, 0.0018347757489638606, 0.00021438116586380412]


In [9]:
report.export_report("povmt_nrep=10_軸上から一番離れた表面の少し内側.pdf",
    estimation_results_list,
    case_name_list,
    estimator_list,
    true_object,
    tester_objects,
    loss_list=loss_list,
    loss_option_list=loss_option_list,
    algo_list=algo_list,
    algo_option_list=algo_option_list,
    seed=seed,
    computation_time=sum(elapsed_times)
)

100%|██████████| 10/10 [00:00<00:00, 10010.27it/s]​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...

​​Generating consictency test blocks ...
** pgdb k=0
** pgdb k=10
** pgdb k=20
** pgdb k=30
** pgdb k=0
** pgdb k=10
** pgdb k=20
** pgdb k=30
** pgdb k=0
** pgdb k=10
** pgdb k=20
** pgdb k=0
** pgdb k=10
** pgdb k=20
** pgdb k=30
​Generating a graph for MSE ...
*** data_dict={'Linear': {'mses': [[0.02500000000000001, 0.002500000000000001, 0.0002500000000000001], [0.025000000000000022, 0.0025000000000000022, 0.0002500000000000002]], 'display_case_names': ['Analytical result (Linear, True)', 'Analytical result (Linear, False)'], 'parameters': [True, False]}, 'LossMinimization': {'mses': [[0.02500000000000001, 0.002500000000000001, 0.0002500000000000001], [0.08, 0.008, 0.0008]], 'display_case_names': ['Analytical result (LossMinimization, True)', 'Analytical result (Lo